In [16]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
import base64
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "username"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input

df = pd.read_csv('/usr/local/datasets/aac_shelter_outcomes.csv')
dff = df.copy()
image_filename = 'Grazioso.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard 1409878_snhu'))),
    html.Div([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'10%', 'width':'10%'})]),
    html.Hr(),
    dcc.Dropdown(id="rescue_choice", 
                options=[
                    {"label": "water_rescue", "value": "water"},
                    {"label": "mountain_rescue", "value":"mountain"},
                    {"label": "disaster_rescue", "value":"disaster"},
                    {"label": "reset", "value":"reset"}],
                 value='water',
                ),
    html.Br(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        row_selectable='single',
    ),
    html.Br(),
     html.Hr(),
     html.Div(
            id='map-id',
            className='col s12 m6',
            ),
    dcc.Graph(id="pie_chart"),
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('datatable-id', 'data'),
    [Input('datatable-id', 'derived_viewport_data'),
     Input('rescue_choice', 'value')])
def update_datatable(data, filtered_choice):
    used_dff = get_filtered_data(filtered_choice)
    
    return used_dff.to_dict('records')
    
@app.callback(
     Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_row_indices):
    
    #dff = pd.DataFrame.from_dict(viewData)
    if not selected_row_indices:
        selected_row_indices = [0]
        
    loc_lat = dff.iloc[selected_row_indices]['location_lat'].iloc[0]
    loc_long = dff.iloc[selected_row_indices]['location_long'].iloc[0]
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[loc_lat, loc_long], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[loc_lat, loc_long], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[selected_row_indices]['name'])
                ])
            ])
        ])
    ]

@app.callback(
     Output('pie_chart', 'figure'),
    [Input('rescue_choice', 'value')])
def update_pie(filter_choice):
    used_dff = get_filtered_data(filter_choice)
       
    figure = px.pie(used_dff, names='breed', color='breed', title="Breed Rescue Chart")
    return figure

# Helper function to build filtered dataframe
def get_filtered_data(filter_choice):
    dff = df.copy() # copy of original
    # Water Rescue filtering
    w_dff = dff[(dff['sex_upon_outcome'] == "Intact Female")] # sex
    w_dff = w_dff[(w_dff['age_upon_outcome_in_weeks'] > 25) & 
                  (w_dff['age_upon_outcome_in_weeks'] < 157)] # age
    w_dff = w_dff[(w_dff["breed"] == "Labrador Retriever Mix") | 
                          (w_dff["breed"] == "Chesapeake Bay Retriever") | 
                         (w_dff["breed"] == "Newfoundland")] # breed
    
    # Mountain Rescue filtering
    m_dff = dff[(dff['sex_upon_outcome'] == "Intact Male")] # sex
    m_dff = m_dff[(m_dff['age_upon_outcome_in_weeks'] > 26)
                  & (m_dff['age_upon_outcome_in_weeks'] < 157)] # age
    m_dff = m_dff[(m_dff['breed'] == "German Shepherd") | 
                  (m_dff['breed'] == "Alaskan Malamute") |
                  (m_dff['breed'] == "Old English Sheepdog") | 
                  (m_dff['breed'] == "Siberian Husky") | 
                  (m_dff['breed'] == "Rottweiler")] # breed
    
    # Disaster Rescue filtering
    d_dff = dff[(dff['sex_upon_outcome'] == "Intact Male")] # sex
    d_dff = d_dff[(d_dff['age_upon_outcome_in_weeks'] > 19) & 
                  (d_dff['age_upon_outcome_in_weeks'] < 301)] # age
    d_dff = d_dff[(d_dff['breed'] == "Doberman Pinscher") |
                  (d_dff['breed'] == "German Shepherd") |
                  (d_dff['breed'] == "Golden Retriever") |
                  (d_dff['breed'] == "Bloodhound") |
                  (d_dff['breed'] == "Rottweiler")] # breed
    

    if filter_choice == "water":
        dff = w_dff
    if filter_choice == "mountain":
        dff = m_dff
    if filter_choice == "disaster":
        dff = d_dff
    if filter_choice == "reset":
        dff = df
    
    return dff


app